Based on https://bambinos.github.io/bambi/notebooks/sleepstudy.html

In [ ]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
az.style.use("arviz-darkgrid")
SEED = 7355608

In [ ]:
data = bmb.load_data("sleepstudy")
data

In [ ]:
def plot_data(data):
    fig, axes = plt.subplots(
        2,
        9,
        figsize=(16, 7.5),
        sharey=True,
        sharex=True,
        dpi=300,
        constrained_layout=False,
    )
    fig.subplots_adjust(left=0.075, right=0.975, bottom=0.075, top=0.925, wspace=0.03)

    axes_flat = axes.ravel()

    for i, subject in enumerate(data["Subject"].unique()):
        ax = axes_flat[i]
        idx = data.index[data["Subject"] == subject].tolist()
        days = data.loc[idx, "Days"].values
        reaction = data.loc[idx, "Reaction"].values

        # Plot observed data points
        ax.scatter(days, reaction, color="C0", ec="black", alpha=0.7)

        # Add a title
        ax.set_title(f"Subject: {subject}", fontsize=14)

    ax.xaxis.set_ticks([0, 2, 4, 6, 8])
    fig.text(0.5, 0.02, "Days", fontsize=14)
    fig.text(0.03, 0.5, "Reaction time (ms)", rotation=90, fontsize=14, va="center")

    return axes

In [ ]:
plot_data(data);

In [ ]:
model = bmb.Model("Reaction ~ 1 + Days + (Days | Subject)", data, categorical="Subject")

In [ ]:
model

In [ ]:
idata = model.fit(draws=2000, random_seed=SEED)

In [ ]:
az.summary(idata, var_names=["Intercept", "Days"], kind="stats")

In [ ]:
az.plot_trace(idata, var_names=["1|Subject", "Days|Subject"]);

In [ ]:
#  extract a subsample from the posterior and stack the chain and draw dims
posterior = az.extract(idata, num_samples=500)

_, ax = plt.subplots()

idata.posterior.plot.scatter(
    x="1|Subject",
    y="Days|Subject",
    hue="Subject__factor_dim",
    add_colorbar=False,
    add_legend=False,
    cmap="tab20",
    edgecolors=None,
)

ax.axhline(c="0.25", ls="--")
ax.axvline(c="0.25", ls="--")
ax.set_xlabel("Subject-specific intercept")
ax.set_ylabel("Subject-specific slope");

In [ ]:
# Obtain the posterior of the mean
model.predict(idata)

# Plot the data
axes = plot_data(data)

# Take the posterior of the mean reaction time
reaction_mean = az.extract(idata)["mu"].values

for subject, ax in zip(data["Subject"].unique(), axes.ravel()):
    idx = data.index[data["Subject"] == subject].tolist()
    days = data.loc[idx, "Days"].values

    # Plot highest density interval / credibility interval
    az.plot_hdi(days, reaction_mean[idx].T[np.newaxis], color="C0", ax=ax)

    # Plot mean regression line
    ax.plot(days, reaction_mean[idx].mean(axis=1), color="C0")